### Cleaning/transforming some objects from pickle and storing them in MongoDB:
db = boxofficemojo, collection = movies

In [75]:
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
import dateutil.parser
import logging
logging.basicConfig(level=logging.DEBUG)
from pymongo import MongoClient

with open("all_boxofficemojo.pkl","r") as picklefile:
    data = pickle.load(picklefile)

In [48]:
print data[:1]

[{'MPAA Rating': u'PG-13', 'openWDG': 247966675.0, 'Production Budget': u'$245 million', 'OWtheaters': u'(#1 rank, 4,134 theaters, $59,982 average)', 'title': u'Star Wars: The Force Awakens (2015)', 'Genre:': u'Sci-Fi Fantasy', 'Domestic Total': None, 'Producer': [u'J.J. Abrams', u'Bryan Burk', u'Kathleen Kennedy'], 'Actor': [u"Lupita Nyong'o", u'John Boyega', u'Daisy Ridley', u'Andy Serkis', u'Harrison Ford', u'Carrie Fisher', u'Mark Hamill', u'Anthony Daniels', u'Peter Mayhew'], 'Director': [u'J.J. Abrams'], 'TLG': 935518389.0, 'Composer': [u'John Williams'], 'Distributor': u'Buena Vista', 'Runtime': u'2 hrs. 16 min.', 'Writer': [u'J.J. Abrams', u'Michael Arndt', u'Lawrence Kasdan'], 'Release Date': u'December 18, 2015'}]


In [49]:
def budget_to_number(budget):
    """ Returns budget as float"""
    
    if budget != 'N/A':
        production_budget = float(budget.strip("$").replace(" million","")) * 1000000
        return production_budget
    else:
        return None
  

In [73]:
def per_theater(theaters):
    """ Returns revenue per theater"""
    
    if '$' in theaters:
        return float(theaters.split(" ")[4].strip("$").replace(",",""))
    else:
        return None
   

In [51]:
def runtime_to_minutes(movie_time):
    """Returns time in minutes"""
    
    rtime = movie_time.split(" ")
    rtime = (float(rtime[0]) * 60. + float(rtime[2]))
    return rtime


In [52]:
def to_date(datestring):
    """Returns date"""
    
    date = dateutil.parser.parse(datestring).date()
    return date

In [82]:
new_l = []

for _ in data:
    
    new_d = {}
    for item in _.items(): 
        try:
            if item[0]  == 'Production Budget':
                new_d[item[0]] = budget_to_number(item[1])
            elif item[0] == 'OWtheaters':
                new_d[item[0]] = per_theater(item[1])
            elif item[0] == 'Runtime':
                new_d[item[0]] = runtime_to_minutes(item[1])
            #elif item[0] == 'Release Date':
                #new_d[item[0]] = to_date(item[1])
            else:
                new_d[item[0]] = item[1]
                
        except Exception:
            pass
        
    new_l.append(new_d)
    
print len(new_l)   
print new_l[:1]

14006
[{'Genre:': u'Sci-Fi Fantasy', 'Production Budget': 245000000.0, 'OWtheaters': 59982.0, 'title': u'Star Wars: The Force Awakens (2015)', 'openWDG': 247966675.0, 'Domestic Total': None, 'Producer': [u'J.J. Abrams', u'Bryan Burk', u'Kathleen Kennedy'], 'Actor': [u"Lupita Nyong'o", u'John Boyega', u'Daisy Ridley', u'Andy Serkis', u'Harrison Ford', u'Carrie Fisher', u'Mark Hamill', u'Anthony Daniels', u'Peter Mayhew'], 'Director': [u'J.J. Abrams'], 'TLG': 935518389.0, 'Composer': [u'John Williams'], 'Distributor': u'Buena Vista', 'MPAA Rating': u'PG-13', 'Writer': [u'J.J. Abrams', u'Michael Arndt', u'Lawrence Kasdan'], 'Release Date': u'December 18, 2015', 'Runtime': 136.0}]


In [83]:
client = MongoClient()

In [84]:
print len(new_l)

14006


In [85]:
db = client.boxofficemojo
movies = db.movies
for _ in new_l:
    movies.save(_)
movies.count()

14006

In [86]:
db.collection_names()

[u'movies', u'system.indexes']